# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from fastcore.test import test_fail
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, List, Callable, Optional
from ecutilities.core import safe_path, path_to_parent_dir, is_type, CurrentMachine

import numpy as np
import pandas as pd
import sys
import subprocess
import warnings

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# System and CLI

In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



# Notebook setup

In [ ]:
#| export
def nb_setup(
    autoreload:bool = True,       # True to set autoreload in this notebook
    paths:List[str|Path] = None   # Paths to add to the path environment variable
    ):
    """Use in first cell of notebook to set autoreload, and add system paths
    
    Always add a path to 'src' in the project root, if the directory exists
    """
    #  Handle paths
    #  Add src if it exists
    nbs_root = path_to_parent_dir('nbs')
    p2src = (nbs_root / '../src').resolve().absolute()
    if p2src.is_dir():
        p = str(p2src.absolute())
        if p not in sys.path:
            sys.path.insert(0, p)
            print(f"Added path: {p2src.absolute()}")
    # Add passed paths
    if paths:
        for p in paths:
            if isinstance(p, Path): p = str(p.resolve().absolute())
            if p not in sys.path:
                sys.path.insert(1, p)
                print(f"Added path: {p}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

In [ ]:
show_doc(nb_setup)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L32){target="_blank" style="float:right; font-size:smaller"}

### nb_setup

>      nb_setup (autoreload:bool=True, paths:List[str|pathlib.Path]=None)

Use in first cell of notebook to set autoreload, and add system paths

Always add a path to 'src' in the project root, if the directory exists

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| autoreload | bool | True | True to set autoreload in this notebook |
| paths | List[str \| Path] | None | Paths to add to the path environment variable |

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          |--- nbs
          |     | --- current_nb.ipynb
          |     | --- ...
          |
          |--- src
          |     | --- module_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

Before running `nb_setup`, `sys.path` does not include the path to the local source directory. After running it, it will be added, unless the directory does not exist.

In [ ]:
sys.path

['/home/vtec/projects/ec-packages/ecutilities/nbs-dev',
 '/home/vtec/miniconda3/envs/ecutils/lib/python310.zip',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/ecutilities']

In [ ]:
nb_setup(autoreload=False)

Added path: /home/vtec/projects/ec-packages/ecutilities/src


In [ ]:
sys.path

['/home/vtec/projects/ec-packages/ecutilities/src',
 '/home/vtec/projects/ec-packages/ecutilities/nbs-dev',
 '/home/vtec/miniconda3/envs/ecutils/lib/python310.zip',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/ecutilities']

We also can add other specific paths:

In [ ]:
path_to_add = str(Path('../nbs').resolve().absolute())
nb_setup(autoreload=False, paths=[path_to_add])

Added path: /home/vtec/projects/ec-packages/ecutilities/nbs


In [ ]:
sys.path

['/home/vtec/projects/ec-packages/ecutilities/src',
 '/home/vtec/projects/ec-packages/ecutilities/nbs',
 '/home/vtec/projects/ec-packages/ecutilities/nbs-dev',
 '/home/vtec/miniconda3/envs/ecutils/lib/python310.zip',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/ecutilities']

In [ ]:
#| export
def install_code_on_cloud(
    package_name:str, # project package name, e.g. metagentools or git+https://github.com/repo.git@main
    quiet:bool=False # install quietly with Trud
):
    """pip install the project code package, when nb is running in the cloud."""
    
    machine = CurrentMachine()

    if machine.is_colab:
        CLOUD = True
        print('The notebook is running on colab.', end=' ')
        print(f'Will install {package_name}')
    elif machine.is_kaggle:
        CLOUD = True
        print('The notebook is running on kaggle.', end=' ')
        print(f'Will install {package_name}')
    elif machine.is_local:
        CLOUD = False
        print('The notebook is running locally, will not automatically install project code')
    else:
        CLOUD = True
        print('The notebook is running on a cloud VM or the machine was not registered as local')
        print(f'Will install {package_name}')

    if CLOUD:
        print(f'Installing project code {package_name}')
        cmd = f"pip install -{'qq' if quiet else ''}U {package_name}"
        run_cli(cmd)
        print((f"{package_name} is installed."))

In [ ]:
show_doc(install_code_on_cloud)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L65){target="_blank" style="float:right; font-size:smaller"}

### install_code_on_cloud

>      install_code_on_cloud (package_name:str, quiet:bool=False)

pip install the project code package, when nb is running in the cloud.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| package_name | str |  | project package name, e.g. metagentools or git+https://github.com/repo.git@main |
| quiet | bool | False | install quietly with Trud |

When using colab, kaggle or another cloud VM, specicif code must be installed every time from the Python Package Index (PyPI) or its GitHub repo.

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
install_code_on_cloud(package_name='metagentools');

The notebook is running locally, will not automatically install project code


# Improve output cell formats

In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L96){target="_blank" style="float:right; font-size:smaller"}

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L104){target="_blank" style="float:right; font-size:smaller"}

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,1.053532,-0.033439,-0.126857,-2.361325,0.859310
1,-2.187972,0.580370,0.043982,-1.831536,0.467534
2,1.116136,-0.556733,0.328271,-1.163368,-0.085929


,0,1,2,3,4,5,6,7,8,9
0,0.527327,0.619668,-0.713957,0.396437,2.380298,-0.349643,-0.250297,0.374774,-0.685882,0.329070
1,0.342080,1.510655,-0.204959,-0.623944,1.092967,1.170887,0.549268,-2.568226,0.525949,0.046807
2,-1.930622,0.766416,-0.858602,-0.908672,0.288908,0.257053,0.814357,0.847788,-1.376114,0.272470


In [ ]:
#| export
class pandas_nrows_ncols:
    """Context manager that sets the max number of rows and cols to apply to any output within the context"""
    def __init__(
        self, 
        nrows:int|None=None, # max number of rows to show; show all rows if `None`
        ncols:int|None=None, # max number of columns to show; show all columns if `None`
    ):
        self.nrows = nrows
        self.ncols = ncols
    
    def __enter__(self):
        self.max_rows = pd.options.display.max_rows
        self.max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = self.nrows
        pd.options.display.max_columns = self.ncols
        return self.max_rows, self.max_cols

    def __exit__(self, exc_type, exc_value, exc_tb):
        pd.options.display.max_rows = self.max_rows
        pd.options.display.max_columns = self.max_cols

In [ ]:
show_doc(pandas_nrows_ncols)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L111){target="_blank" style="float:right; font-size:smaller"}

### pandas_nrows_ncols

>      pandas_nrows_ncols (nrows:int|None=None, ncols:int|None=None)

Context manager that sets the max number of rows and cols to apply to any output within the context

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| nrows | int \| None | None | max number of rows to show; show all rows if `None` |
| ncols | int \| None | None | max number of columns to show; show all columns if `None` |

With no context manager, the pandas object are displayed with a maximum of 60 rows and 20 columns.

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,48,57,43,75,12,71,70,91,7,50,...,54,43,82,25,94,69,62,46,20,83
1,73,31,15,20,90,60,76,94,3,35,...,48,22,96,64,70,14,64,9,9,80
2,16,42,4,18,70,18,3,85,97,97,...,54,20,28,66,23,20,87,88,26,60


Using the context manager, all rows and columns will be displayed

In [ ]:
with pandas_nrows_ncols():
    display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,48,57,43,75,12,71,70,91,7,50,24,10,95,94,3,16,83,85,63,30,66,66,35,44,64,82,90,25,67,65,7,58,94,21,22,67,94,71,44,76,54,43,82,25,94,69,62,46,20,83
1,73,31,15,20,90,60,76,94,3,35,98,27,24,85,30,2,21,45,15,94,62,74,78,0,45,97,9,13,47,61,30,52,72,59,36,81,83,91,78,46,48,22,96,64,70,14,64,9,9,80
2,16,42,4,18,70,18,3,85,97,97,15,83,26,59,53,0,81,88,28,23,81,98,49,61,88,33,51,52,97,15,46,34,22,73,84,94,86,93,11,61,54,20,28,66,23,20,87,88,26,60


It is also possible to specifically define the number of rows and columns to display

In [ ]:
with pandas_nrows_ncols(nrows=2, ncols=6):
    display(df)

,0,1,2,...,47,48,49
0,48,57,43,...,46,20,83
...,...,...,...,...,...,...,...
2,16,42,4,...,88,26,60


In [ ]:
with pandas_nrows_ncols(2,6):
    print(df)

    0   1   2   ...  47  48  49
0   48  57  43  ...  46  20  83
..  ..  ..  ..  ...  ..  ..  ..
2   16  42   4  ...  88  26  60

[3 rows x 50 columns]


> ### Technical background
> 
> the context manager uses pandas's [`options API`](https://pandas.pydata.org/docs/user_guide/options.html)

In [ ]:
pd.options.display.max_rows, pd.options.display.max_columns

(60, 20)

In [ ]:
pd.get_option('display.max_rows'), pd.get_option('display.max_columns')

(60, 20)

In [ ]:
pd.describe_option('display.max_rows')

display.max_rows : int
    If max_rows is exceeded, switch to truncate view. Depending on
    `large_repr`, objects are either centrally truncated or printed as
    a summary view. 'None' value means unlimited.

    In case python/IPython is running in a terminal and `large_repr`
    equals 'truncate' this can be set to 0 and pandas will auto-detect
    the height of the terminal and print a truncated object which fits
    the screen height. The IPython notebook, IPython qtconsole, or
    IDLE do not run in a terminal and hence it is not possible to do
    correct auto-detection.
    [default: 60] [currently: 60]


In [ ]:
pd.options.display.max_rows = 10
pd.reset_option('display.max_rows')
pd.options.display.max_rows

60

In [ ]:
#| export
def display_full_df(
    df:pd.DataFrame|pd.Series,  # `DataFrame` or `Series` to display
):
    """Display a pandas `DataFrame` or `Series` showing all rows and columns"""
    if is_type(df, pd.DataFrame, raise_error=False) or is_type(df, pd.Series, raise_error=False):
        with pandas_nrows_ncols():
            display(df)
    else:
        raise TypeError(f"df must me a pandas `DataFrame` or `Series`, not a {type(df)}")

In [ ]:
show_doc(display_full_df)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L145){target="_blank" style="float:right; font-size:smaller"}

### display_full_df

>      display_full_df
>                       (df:pandas.core.frame.DataFrame|pandas.core.series.Serie
>                       s)

Display a pandas `DataFrame` or `Series` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame \| pd.Series | `DataFrame` or `Series` to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,74,75,67,32,87,39,30,47,77,48,...,10,73,42,77,99,24,98,65,62,63
1,40,36,70,54,5,26,54,10,33,68,...,29,12,84,51,87,15,3,40,87,3
2,53,62,95,85,99,56,94,23,20,48,...,81,13,14,20,88,45,77,73,95,54


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,74,75,67,32,87,39,30,47,77,48,38,14,76,76,18,74,62,52,75,0,7,46,61,45,99,93,34,6,53,64,56,48,72,48,53,46,63,45,78,58,10,73,42,77,99,24,98,65,62,63
1,40,36,70,54,5,26,54,10,33,68,51,81,54,12,34,53,73,74,6,50,11,26,78,45,77,3,78,45,9,17,66,17,95,92,12,22,4,5,52,64,29,12,84,51,87,15,3,40,87,3
2,53,62,95,85,99,56,94,23,20,48,69,59,2,55,38,89,2,38,34,38,87,98,64,69,98,79,62,20,61,62,28,12,8,47,0,50,87,18,21,47,81,13,14,20,88,45,77,73,95,54


In [ ]:
#| hide
display_full_df(df.loc[0, :].T)

0     74
1     75
2     67
3     32
4     87
5     39
6     30
7     47
8     77
9     48
10    38
11    14
12    76
13    76
14    18
15    74
16    62
17    52
18    75
19     0
20     7
21    46
22    61
23    45
24    99
25    93
26    34
27     6
28    53
29    64
30    56
31    48
32    72
33    48
34    53
35    46
36    63
37    45
38    78
39    58
40    10
41    73
42    42
43    77
44    99
45    24
46    98
47    65
48    62
49    63
Name: 0, dtype: int64

In [ ]:
msg = 'should raise a TypeError'
contains = 'df must me a pandas `DataFrame` or `Series`'

test_fail(display_full_df, args=['a string'], msg=msg, contains=contains)

In [ ]:
#| hide
nbdev_export()